In [1]:
from Utils.Toolkit import get_static_data
import numpy as np
import scipy.sparse as sps

data = get_static_data(5)

In [2]:
train = data['train']
test = data['test']
ucm = data['UCM'].tocsr()

print(type(ucm))
print(type(test))
print(type(train))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>


# Maurizio Hyb Recommender

In [ ]:
from Algorithms.HYB.HybridRecommender import HybridRecommender

hybrid = HybridRecommender(train.copy(), ucm.copy())
hybrid.fit(weight_itemcf=1.0, weight_p3=1.0, weight_rp3=1.0, weight_als=1.0, weight_slimel=0.0)

# My Hybrid Recommender

In [ ]:
from Recommenders.HYB.hyb import HYB

w = {
    'item_cf': 1,
    'ALS': 1,
    'P3Alpha': 1,
    'RP3Alpha': 1
}

userCBF_args = {
    'topK' : 1000,
    'shrink' : 7950
}

P3alpha_args = {
    'topK': 66,
    'alpha': 0.2731573847973295,
    'normalize': True
}

itemCF_args = {
    'topK': 12,
    'shrink': 88,
    'similarity': 'tversky',
    'normalize': True,
    'fw': 'none',
    'tversky_alpha': 0.12331166243379268,
    'tversky_beta': 1.9752288743799558,
    'asymmetric_alpha': 0.0
}

my_hyb = HYB(weights=[w,w,w], itemCF_args=itemCF_args, userCBF_args=userCBF_args, P3alpha_args=P3alpha_args)
my_hyb.fit(train.copy(), ucm.copy())

In [ ]:
list(my_hyb.recommend(0))

In [ ]:
hybrid.recommend(0)[:10]

### Results are different, why?
Let's analyse single recommenders

# Item CFs

In [ ]:
my_hyb_item = my_hyb.itemCF
hybrid_item = hybrid.itemCF

In [ ]:
in_dif = []
r1 = hybrid_item.recommend(0)
r2 = my_hyb_item.recommend(0)
r1 = np.asarray(r1).ravel()

print(r1[:10])
print(r2)

### Differencies solved

# P3 CFs

In [ ]:
my_hyb_p3 = my_hyb.P3alpha
hybrid_p3 = hybrid.P3

In [ ]:
d = []

for i in range(30911):
    r2 = hybrid_p3._compute_item_score(i).ravel()
    r1 = my_hyb_p3.get_expected_ratings(i)

    if not np.array_equal(r1, r2):
        print("False")
d

In [ ]:
d = []
r2val = 14980
r1val = 8028

for i in range(30911):
    r2 = hybrid_p3.recommend(0)
    r1 = my_hyb_p3.recommend(0, 18945)

    if not np.array_equal(r1, r2):
        print(i)
        break

r2_ord = r2
r1_ord = r1

In [ ]:
r2 = hybrid_p3._compute_item_score(0).ravel()[r2_ord]
r1 = my_hyb_p3.get_expected_ratings(0)[r2_ord]

print(r2[0:10])
print(r1[0:10])

### This differs too!

# RP3

In [ ]:
my_hyb_rp3 = my_hyb.RP3alpha
hybrid_rp3 = hybrid.RP3

In [ ]:
r1 = hybrid_rp3.recommend(1231, 10000)
r2 = my_hyb_rp3.recommend(1231, 10000)

np.array_equal(r1, r2)

### Ok this is strange, this is still different

# userCBF

In [ ]:
my_hyb_ucbf = my_hyb.userCBF
hybrid_ucbf = hybrid.userCBF

In [ ]:
r1 = hybrid_ucbf._compute_item_score(0).ravel()
r2 = my_hyb_ucbf.RM[0].toarray().ravel()

### This can't be, everyone is different :/

# HYB

In [ ]:
my_hyb.recommend(0)

In [ ]:
hybrid.recommend(0, 10)

# ---------------------------------------------------

# Item Collaborative Filtering

In [ ]:
from Algorithms.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.CF.item_cf import ItemBasedCollaborativeFiltering

data = get_static_data(5)
train = data['train']

itemCF = ItemKNNCFRecommender(train)
itemCF.fit(12, 88, similarity='tversky', normalize=True,
               feature_weighting='none', tversky_alpha=0.12331166243379268, tversky_beta=1.9752288743799558, asymmetric_alpha=0.0)

my_itemCF = ItemBasedCollaborativeFiltering(topK=12,
                                            shrink=88,
                                             feature_weighting='none',
                                             similarity='tversky',
                                             tversky_alpha=0.12331166243379268,
                                             tversky_beta=1.9752288743799558,
                                             asymmetric_alpha=0.0)
my_itemCF.fit(train)

In [ ]:
different = []

for i in range(len(itemCF.W_sparse.data)):
    if itemCF.W_sparse.data[i] != my_itemCF.SM_item.data[i]:
        different.append(i)
    if itemCF.W_sparse.indices[i] != my_itemCF.SM_item.indices[i]:
        different.append(i)

print('There are {} elements differing'.format(len(different)))

In [ ]:
diff = []

for user_id in range(30911):    
    score1 = itemCF._compute_item_score(user_id).ravel()
    score2 = my_itemCF.RM[user_id].toarray().ravel()

    if not np.array_equal(score1, score2):
        diff.append(user_id)

diff[0:10]

In [ ]:
RM_item = itemCF._compute_item_score(np.arange(30911))
RM_item

In [ ]:
RM = my_itemCF.RM.toarray()
RM

In [ ]:
user_id = 4

first_user1 = RM[user_id]
first_user1[first_user1.nonzero()]

In [ ]:
first_user2 = RM_item[user_id]
first_user2[first_user2.nonzero()]

In [ ]:
diff_index = []

for i in range(len(first_user2[first_user2.nonzero()])):
    if first_user1[first_user1.nonzero()][i] != first_user2[first_user2.nonzero()][i]:
        diff_index.append(i)

diff_index

In [ ]:
type(first_user2[first_user2.nonzero()][3])

In [ ]:
type(first_user1[first_user1.nonzero()][3])

# P3Alpha

In [ ]:
from Algorithms.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.Graph.P3GraphRecommender import P3AlphaRecommender

data = get_static_data(10)
train = data['train']

my_p3 = P3AlphaRecommender(topK=100, alpha=1.0, min_rating=0, implicit=True, normalize_similarity=False)
p3 = P3alphaRecommender(train)
p3.fit(topK=100, alpha=1.0, min_rating=0, implicit=True, normalize_similarity=False)
my_p3.fit(train)

In [ ]:
print(len(p3.recommend(3)))
print(len(my_p3.recommend(3, 19000)))

In [ ]:
for user_in in range(30911):
    if not np.array_equal(my_p3.recommend(user_id, 19000), p3.recommend(user_id)):
        print(user_id)
        break

In [3]:
from Algorithms.KNN.UserKNNCBFRecommender import UserKNNCBFRecommender
from Recommenders.CBF.user_CBF import UserContentBasedRecommender

my_ucbf = UserContentBasedRecommender(topK=10, shrink=10)
ucbf = UserKNNCBFRecommender(train, ucm)

ItemKNNCBFRecommender: URM Detected 8392 (27.15 %) cold users.
ItemKNNCBFRecommender: URM Detected 3486 (18.85 %) cold items.


In [4]:
my_ucbf.fit(train, ucm)
ucbf.fit(topK=10, shrink=10, similarity='cosine', feature_weighting='TF-IDF')

In [5]:
list(my_ucbf.recommend(3))

[18494, 6068, 6160, 6161, 6162, 6163, 6164, 6165, 6166, 6167]

In [6]:
ucbf.recommend(3)[:10]

[18494, 6068, 6160, 6161, 6162, 6163, 6164, 6165, 6166, 6167]

In [77]:
list_ = [None] * 30911
score = ucbf._compute_item_score([5, 6])
list_[0] = ucbf._compute_item_score(5).ravel().tolist()
list_

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [62]:
my_score = my_ucbf.get_expected_ratings(5)
my_score[my_score>0] * 1

array([1.47598493, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 1.47598493, 1.47598493,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799

In [67]:
score = (my_score * 0) + score
score[score > 0]

array([1.47598493, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 1.47598493, 1.47598493,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799247,
       0.73799247, 0.73799247, 0.73799247, 0.73799247, 0.73799

In [ ]:
for user_id in range(30911):
    if not np.array_equal(my_ucbf.recommend(user_id, 19000), ucbf.recommend(user_id)):
        print(i)
        break

In [11]:
rec_list = ucbf.recommend([0, 1, 2, 3, 4, 5], cutoff=10)
rec_list

[[18494, 6068, 6160, 6161, 6162, 6163, 6164, 6165, 6166, 6167],
 [18494, 6068, 6160, 6161, 6162, 6163, 6164, 6165, 6166, 6167],
 [18494, 6068, 6160, 6161, 6162, 6163, 6164, 6165, 6166, 6167],
 [18494, 6068, 6160, 6161, 6162, 6163, 6164, 6165, 6166, 6167],
 [18494, 6068, 6160, 6161, 6162, 6163, 6164, 6165, 6166, 6167],
 [12469, 194, 6526, 6528, 3876, 12763, 4722, 11964, 10914, 12083]]

In [38]:
fix = []
user_id = 0

for list_ in rec_list:
    new = [user_id, np.squeeze(np.array(list_))]
    fix.append(new)
    user_id += 1

import pandas as pd

cols = ['user_id', 'items_id']

df = pd.DataFrame(fix, columns=cols)
df

,user_id,items_id
0,0,"[18494, 6068, 6160, 6161, 6162, 6163, 6164, 61..."
1,1,"[18494, 6068, 6160, 6161, 6162, 6163, 6164, 61..."
2,2,"[18494, 6068, 6160, 6161, 6162, 6163, 6164, 61..."
3,3,"[18494, 6068, 6160, 6161, 6162, 6163, 6164, 61..."
4,4,"[18494, 6068, 6160, 6161, 6162, 6163, 6164, 61..."
5,5,"[12469, 194, 6526, 6528, 3876, 12763, 4722, 11..."


In [48]:
string = df.to_csv(index=False)
string = string.replace(']', '')
string = string.replace('[', '')
string

'user_id,items_id\n0,18494  6068  6160  6161  6162  6163  6164  6165  6166  6167\n1,18494  6068  6160  6161  6162  6163  6164  6165  6166  6167\n2,18494  6068  6160  6161  6162  6163  6164  6165  6166  6167\n3,18494  6068  6160  6161  6162  6163  6164  6165  6166  6167\n4,18494  6068  6160  6161  6162  6163  6164  6165  6166  6167\n5,12469   194  6526  6528  3876 12763  4722 11964 10914 12083\n'

In [50]:
%%time
text_file = open("out.csv", "w")
n = text_file.write(string)
text_file.close()

CPU times: user 504 µs, sys: 903 µs, total: 1.41 ms
Wall time: 604 µs


# Performance test

In [52]:
%%time
rec_list = ucbf.recommend(np.arange(30911), cutoff=10)

fix = []
user_id = 0

for list_ in rec_list:
    new = [user_id, np.squeeze(np.array(list_))]
    fix.append(new)
    user_id += 1

import pandas as pd

cols = ['user_id', 'items_id']

df = pd.DataFrame(fix, columns=cols)

string = df.to_csv(index=False)
string = string.replace(']', '')
string = string.replace('[', '')

text_file = open("out.csv", "w")
n = text_file.write(string)
text_file.close()

CPU times: user 19.1 s, sys: 4.81 s, total: 23.9 s
Wall time: 24.3 s
